## Homework 2
by Quynh Anh Pham

In [9]:
import pandas as pd
import numpy as np
import requests

1. Get the time series for the following cryptocurrencies from `cryptowat.ch` starting from 2021-11-22, hourly data
    1. ETH
    2. SOL
    3. AVAX
    4. USDT
    5. FLOW

In [10]:
def get_data(token, start_date, exchange='coinbase-pro'):
    res = requests.get(
        f'https://api.cryptowat.ch/markets/{exchange}/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp(start_date).timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df


In [11]:
cb_tokens = ['ETH', 'SOL', 'AVAX', 'USDT']
kk_tokens = ['FLOW']
start_date = '2021-11-22'

tokens = cb_tokens + kk_tokens

In [12]:
df_cb = [get_data(token,start_date) for token in cb_tokens]
df_kk = [get_data(token,start_date,'kraken') for token in kk_tokens]

In [13]:
df_base = df_cb + df_kk

In [14]:
df_base

[                     ts     open     high      low    close        volume  \
 0   2021-11-22 00:00:00  4317.98  4342.24  4246.07  4262.99   7262.562789   
 1   2021-11-22 01:00:00  4263.04  4270.34  4212.45  4234.37   8437.946084   
 2   2021-11-22 02:00:00  4234.87  4246.72  4171.17  4217.89   9259.725370   
 3   2021-11-22 03:00:00  4217.88  4223.48  4163.58  4193.47   9259.899519   
 4   2021-11-22 04:00:00  4192.95  4213.59  4147.00  4168.35   7934.546906   
 ..                  ...      ...      ...      ...      ...           ...   
 685 2021-12-20 13:00:00  3814.63  3815.80  3770.17  3782.86   4682.198085   
 686 2021-12-20 14:00:00  3782.85  3811.00  3766.88  3776.87   5356.147003   
 687 2021-12-20 15:00:00  3777.07  3848.53  3754.80  3840.19  17507.992019   
 688 2021-12-20 16:00:00  3838.73  3842.00  3794.17  3801.41   6336.438977   
 689 2021-12-20 17:00:00  3801.69  3810.23  3792.71  3806.83   1287.000212   
 
         volumeUSD token  
 0    3.123077e+07   ETH  
 1    3.

In [15]:
df = pd.concat(df_base, keys='token').set_index('ts')

df.head()

,open,high,low,close,volume,volumeUSD,token
ts,,,,,,,
2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH
2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH
2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH
2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH
2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH


2. Get the total USD volume traded for each token in a dataframe, sorted from highest volume to lowest volume

In [ ]:
df_base.groupby('token')['volumeUSD'].sum().to_frame().sort_values('volumeUSD',ascending=False)

3. Add a column that calculates the close price ratio between ETH and SOL for each hour (i.e. close price of ETH / close price of SOL for each period)

In [ ]:
# adding the ratio column to the existing data frame
df['eth_sol_close_ratio'] = df[df['token']=='ETH']['close'] / df[df['token'] == 'SOL']['close']

In [ ]:
# removing the values from every tokens other than ETH
df['eth_sol_close_ratio'].loc[df.token != "ETH"] = np.nan

In [ ]:
df[df.token == "ETH"]

In [ ]:
df[df.token != "ETH"]

In [ ]:
# Another view

eth_sol = df[df.token=="ETH"]["close"].rename("close_ETH").to_frame().join(
    df[df.token=="SOL"]["close"].rename("close_SOL").to_frame()
)
eth_sol['close_ratio']= eth_sol['close_ETH']/eth_sol['close_SOL']
eth_sol

4. Change the name of the `volume` and `volumeUSD` columns to `volumeBase` and `volumeTerm`

In [ ]:
df.rename(
columns={
    "volume":"volumeBase",
    "volumeUSD": "volumeTerm"
}
)

5. Create a fat table indexed by the timestamp, and each column is the close price of each token (i.e. this should be a table of  200 rows and 5 columns)


In [ ]:
# not sure why this is not working 
# df_fat = pd.DataFrame()

for tok in tokens:
    cls_coin = df[df.token == tok]['close'].rename(f'close_{tok}').to_frame()
    if tok == tokens[0]:
        df_fat = cls_coin
    df_fat.merge(cls_coin, on='ts', how='left')
#     print(df_fat)

In [ ]:
df_fat.head()

In [ ]:
# redo another method

def clean_dataframe(original_df):
    cleaned_df = original_df.set_index('ts')['close'].rename(f'close_{original_df.token[0]}').to_frame()
    return cleaned_df 



In [ ]:
cleaned_dataframes = [clean_dataframe(original_df) for original_df in df_base]

In [ ]:
import functools

In [ ]:
df_fat = functools.reduce(lambda a,b: a.join(b), cleaned_dataframes)
df_fat

6. Calculate the hour by hour log return of the close price of each token (return is calculated by np.log(price_t / price_{t-1}))


In [ ]:
df_returns = pd.DataFrame()

for tok in tokens:
    df_returns[f'log_returns_{tok}'] = np.log(df_fat[f'close_{tok}']/df_fat[f'close_{tok}'].shift(1))
    
df_returns


[Stretch] calculate the correlation of the tokens using the table above


In [ ]:
df_returns.corr()


[Stretch] visualize the correlation in a matplpotlib plot


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.matshow(df_returns.corr())
plt.show()